# **3.2.1 어텐션 가중치 시각화**

In [1]:
!git clone https://github.com/rickiepark/nlp-with-transformers.git
%cd nlp-with-transformers
from install import *
install_requirements(chapter=3)

Cloning into 'nlp-with-transformers'...
remote: Enumerating objects: 653, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 653 (delta 47), reused 15 (delta 8), pack-reused 569
Receiving objects: 100% (653/653), 62.41 MiB | 14.97 MiB/s, done.
Resolving deltas: 100% (335/335), done.
/content/nlp-with-transformers
⏳ Installing base requirements ...
✅ Base requirements installed!
Using transformers v4.35.2
Using datasets v2.16.1
Using accelerate v0.26.1
Using sentencepiece v0.1.99
Using bertviz
No GPU was detected! This notebook can be *very* slow without a GPU 🐢
Go to Runtime > Change runtime type and select a GPU hardware accelerator.


In [2]:
 # neuron_view 모듈 사용
from transformers import AutoTokenizer
from bertviz.transformers_neuron_view import BertModel
from bertviz.neuron_view import show

#Bertviz의 모델 클래스 객체 초기화
model_ckpt = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = BertModel.from_pretrained(model_ckpt)
text = "time flies like an arrow"

# 특정 인코더 층과 어텐션 헤드에 대한 시각화 생성
show(model, "bert", tokenizer, text, display_mode="light", layer=0, head=8) #

Output hidden; open in https://colab.research.google.com to view.

#**트랜스포머 아키텍처 구현**

##**1. 텍스트 토큰화**

In [3]:
inputs = tokenizer(text, return_tensors="pt", add_special_tokens=False)
inputs.input_ids

tensor([[ 2051, 10029,  2066,  2019,  8612]])

- 시퀀스에 있는 각 토큰은 토크나이저 어휘사전에서 고유한 각 ID에 매핑됨
- add_special_tokens=False 로 두어 [CLS], [SEP] 토큰 제외

##**2. 밀집 임베딩 만들기**

밀집(dense)은 임베딩에 있는 모든 원소의 값이 0이 아니라는 의미임

+) 추가로 원-핫 인코딩은 sparse함 > 한 원소를 제외한 모든 원소가 0이기 때문

파이토치의 torch.nn.Embedding 층은 각 입력 ID에 대해 룩업테이블처럼 동작

+) 룩업테이블 : 자주 사용하는 데이터 값들을 정리해 놓은 표

In [4]:
from torch import nn
from transformers import AutoConfig

# AutoConfig 클래스를 사용해 bert-base-uncased 체크포인트에 관한 config.json 파일 로드
config = AutoConfig.from_pretrained(model_ckpt)
token_emb = nn.Embedding(config.vocab_size, config.hidden_size)
token_emb

Embedding(30522, 768)

- 허깅페이스 트랜스포머스에서 모든 ckpt는 vocab_size, hidden_size 등과 같은
    
    다양한 하이퍼파라미터가 지정된 설정 파일이 할당됨


    
- 이 경우 입력 ID가 nn.Embedding에 저장된 30,522개 임베딩 벡터 중 하나에 매핑되고,
    
    각 벡터의 크기는 768이다
    


- AutoConfig 클래스는 모델 예측 포맷을 지정하는데 사용되는 추가적인 메타데이터도 저장




+) 메타 데이터 : 데이터에 대한 데이터. 메타는 한 단계 더 위에 있는 것을 가리키는 말

입력 ID 전달해 임베딩 만들기

In [5]:
inputs_embeds = token_emb(inputs.input_ids)
inputs_embeds.size()

torch.Size([1, 5, 768])

이 층은 [batch_size, seq_len, hidden_dim] 크기의 텐서를 출력

#**어텐션 층**

문맥과 독립적인 토큰 임베딩을 혼합해 의미를 명확하게 하고 토큰 표현에 문맥 내용을 주입

##**1. 어텐션 점수 계산**


Q, K, V 벡터를 만들고 점곱을 유사도 함수로 사용해 어텐션 점수 계산하기

In [6]:
import torch
from math import sqrt

query = key = value = inputs_embeds
dim_k = key.size(-1)

# torch.bmm() 은 배치 행렬 - 행렬 곱셈
scores = torch.bmm(query, key.transpose(1,2)) / sqrt(dim_k)
scores.size()

torch.Size([1, 5, 5])

torch.bmm 함수는 행렬 곰셈을 수행해서 크기가 [batch_size, seq_len, hidden_dim]인 쿼리와 키 벡터의 어텐션 점수 계산을 단순화한다.

- 배치에 있는 샘플마다 5 x 5 크기의 어텐션 점수 행렬이 만들어진다



- 추후 임베딩에 독립적인 가중치 행렬 W를 각각 적용해 쿼리, 키 값 벡터 생성할 것



- 여기서는 간단하게 모두의 값을 같게 하였음

스케일드 점곱 어텐션에서 점곱은 임베딩 벡터의 크기로 스케일을 조정

이는 훈련 도중 큰 수의 빈번한 발생을 줄여 소프트 맥스 함수의 포화를 방지한다

##**2. 소프트맥스 함수 적용하기**

In [7]:
import torch.nn.functional as F

weights = F.softmax(scores, dim=-1)
weights.sum(dim=-1)

tensor([[1., 1., 1., 1., 1.]], grad_fn=<SumBackward1>)

##**3. 어텐션 가중치 곱하기**

In [8]:
attn_outputs = torch.bmm(weights, value)
attn_outputs.shape

torch.Size([1, 5, 768])

추후 사용을 위해 하나의 함수로 만들기

In [9]:
def scaled_dot_product_attention(query, key, value):
  dim_k = query.size(-1)
  scores = torch.bmm(query, key.transpose(1, 2)) / sqrt(dim_k)
  weights = F.softmax(scores, dim=-1)
  return torch.bmm(weights, value)

#**멀티 헤드 어텐션**

세 개의 선형 투영을 사용해 초기 토큰 벡터를 세 개의 공간에 투영하는 식으로 쿼리, 키, 값 벡터 다르게 만들기

실제로 셀프 어텐션 층은 각 임베딩에 독립적인 선형 변환 세 개를 적용해 쿼리, 키, 값 벡터 생성



이런 변환은 임베딩을 투영하며 각 투영은 학습 가능한 파라미터를 갖는다



따라서 셀프 어텐션 층은 시퀀스의 다양한 의미에 초점을 맞춘다

##**1. 어텐션 헤드**

각 선형 투영 집합

In [10]:
# 단일 어텐션 헤드
class AttentionHead(nn.Module):
  def __init__(self, embed_dim, head_dim):
    super().__init__()

    # 세 개의 독립된 선형층 생성
    self.q = nn.Linear(embed_dim, head_dim)
    self.k = nn.Linear(embed_dim, head_dim)
    self.v = nn.Linear(embed_dim, head_dim)

  def forward(self, hidden_state):
    attn_outputs = scaled_dot_product_attention(
        self.q(hidden_state), self.k(hidden_state), self.v(hidden_state))
    return attn_outputs

세 개의 선형층은 임베딩 벡터에 행렬 곱셈을 적용해 [batch_size, seq_len, head_dim] 크기의 텐서를 만든다

- head_dim : 투영하려는 차원의 크기


head_dim이 토큰의 임베딩 차원(embed_dim)보다 더 작을 필요는 없지만,  
실전에서는 헤드마다 계산이 일정하도록 embed_dim과 배수가 되게 선택한다

##**2. 멀티 헤드 어텐션 층 만들기**

In [11]:
class MultiHeadAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        embed_dim = config.hidden_size
        num_heads = config.num_attention_heads
        head_dim = embed_dim // num_heads
        self.heads = nn.ModuleList(
            [AttentionHead(embed_dim, head_dim) for _ in range(num_heads)]
        )
        self.output_linear = nn.Linear(embed_dim, embed_dim)

    def forward(self, hidden_state):
        x = torch.cat([h(hidden_state) for h in self.heads], dim=-1)
        x = self.output_linear(x)
        return x

멀티 헤드 어텐션 층이 기대하는 입력 크기를 만드는지 확인하기

MultiHeadAttention 모듈을 초기화할 때 앞서 사전 훈련된 BERT 모델에서 로드한 설정 전달

> BERT와 동일한 설정을 사용

In [12]:
multihead_attn = MultiHeadAttention(config)
attn_output = multihead_attn(inputs_embeds)
attn_output.size()

torch.Size([1, 5, 768])

In [13]:
from bertviz import head_view
from transformers import AutoModel

model = AutoModel.from_pretrained(model_ckpt, output_attentions=True)

sentence_a = "time flies like an arrow"
sentence_b = "fruit flies like a banana"

viz_inputs = tokenizer(sentence_a, sentence_b, return_tensors='pt')
attention = model(**viz_inputs).attentions
sentence_b_start = (viz_inputs.token_type_ids == 0).sum(dim=1)
tokens = tokenizer.convert_ids_to_tokens(viz_inputs.input_ids[0])

head_view(attention, tokens, sentence_b_start, heads=[8])

Output hidden; open in https://colab.research.google.com to view.

#**피드 포워드 층**

In [14]:
class FeedForward(nn.Module):
  def __init__(self, config):  # config : 학습에 필요한 파라미터들을 정의해주는 파일
    super().__init__()
    self.linear_1 = nn.Linear(config.hidden_size, config.intermediate_size)
    self.linear_2 = nn.Linear(config.intermediate_size, config.hidden_size)
    self.gelu = nn.GELU()
    self.dropout = nn.Dropout(config.hidden_dropout_prob)

  def forward(self, x):
    x = self.linear_1(x)
    x = self.gelu(x)
    x = self.linear_2(x)
    x = self.dropout(x)
    return x

In [15]:
# 어텐션 출력을 전달해 테스트하기
feed_forward = FeedForward(config)
ff_outputs = feed_forward(attn_outputs)
ff_outputs.size()

torch.Size([1, 5, 768])

#**층 정규화 추가하기**

사전 층 정규화를 이용한 코드

In [16]:
class TransformerEncoderLayer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.layer_norm_1 = nn.LayerNorm(config.hidden_size)
        self.layer_norm_2 = nn.LayerNorm(config.hidden_size)
        self.attention = MultiHeadAttention(config)
        self.feed_forward = FeedForward(config)

    def forward(self, x):
        # 층 정규화를 적용하고 입력을 쿼리, 키, 값으로 복사한다
        hidden_state = self.layer_norm_1(x)
        # 어텐션에 스킵 연결을 적용한다
        x = x + self.attention(hidden_state)
        # 스킵 연결과 피드 포워드 층을 적용한다
        x = x + self.feed_forward(self.layer_norm_2(x))
        return x

In [17]:
# 입력 임베딩으로 테스트하기
encoder_layer = TransformerEncoderLayer(config)
inputs_embeds.shape, encoder_layer(inputs_embeds).size()

(torch.Size([1, 5, 768]), torch.Size([1, 5, 768]))

#**위치 임베딩**

간단하지만 효과적.

In [18]:
class Embeddings(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.token_embeddings = nn.Embedding(config.vocab_size,
                                             config.hidden_size)
        self.position_embeddings = nn.Embedding(config.max_position_embeddings,
                                                config.hidden_size)
        self.layer_norm = nn.LayerNorm(config.hidden_size, eps=1e-12)
        self.dropout = nn.Dropout()

    def forward(self, input_ids):
        # 입력 시퀀스에 대해 위치 ID를 만들기
        seq_length = input_ids.size(1)
        position_ids = torch.arange(seq_length, dtype=torch.long).unsqueeze(0)
        # 토큰 임베딩과 위치 임베딩 만들기
        token_embeddings = self.token_embeddings(input_ids)
        position_embeddings = self.position_embeddings(position_ids)
        # 토큰 임베딩과 위치 임베딩 합치기
        embeddings = token_embeddings + position_embeddings
        embeddings = self.layer_norm(embeddings)
        embeddings = self.dropout(embeddings)
        return embeddings


In [19]:
embedding_layer = Embeddings(config)
embedding_layer(inputs.input_ids).size()

torch.Size([1, 5, 768])

임베딩과 인코더 층을 연결해 완전한 트랜스포머 인코더 만들기

In [20]:
class TransformerEncoder(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.embeddings = Embeddings(config)
    self.layers = nn.ModuleList([TransformerEncoderLayer(config)
                                for _ in range(config.num_hidden_layers)])

  def forward(self, x):
    x = self.embeddings(x)
    for layer in self.layers:
      x = layer(x)
    return x

In [21]:
# 인코더의 출력 크기 확인하기
encoder = TransformerEncoder(config)
encoder(inputs.input_ids).size()

torch.Size([1, 5, 768])

#**분류 헤드 추가하기**

In [22]:
class TransformerForSequenceClassification(nn.Module):
    def __init__(self, config):
        super().__init__() # 부모 클래스의 속성 및 메소드를 가져오는 것
        self.encoder = TransformerEncoder(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, x):
        x = self.encoder(x)[:, 0, :] # [CLS] 토큰의 은닉 상태를 선택
        x = self.dropout(x)
        x = self.classifier(x)
        return x

In [23]:
config.num_labels = 3
encoder_classifier = TransformerForSequenceClassification(config)
encoder_classifier(inputs.input_ids).size()

torch.Size([1, 3])

#**디코더**

##**마스크드 셀프 어텐션**

In [24]:
# 파이토치 tril() 함수 이용하여 하삼각행렬 만들기
seq_len = inputs.input_ids.size(-1)
mask = torch.tril(torch.ones(seq_len, seq_len)).unsqueeze(0)
mask[0]

tensor([[1., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1.]])

Tensor.maseked_fill()을 사용해 0을 음의 무한대로 바꾸면 어텐션 헤드가 미래 토큰을 보지 못함

In [25]:
scores.masked_fill(mask == 0, -float("inf"))

tensor([[[ 2.8436e+01,        -inf,        -inf,        -inf,        -inf],
         [ 1.9184e+00,  2.6804e+01,        -inf,        -inf,        -inf],
         [-1.1337e+00, -1.2506e+00,  2.7716e+01,        -inf,        -inf],
         [ 1.0465e+00, -3.7238e-01, -5.6484e-01,  2.8508e+01,        -inf],
         [-2.4737e+00, -1.8910e+00,  1.0783e+00, -3.1022e-03,  2.8363e+01]]],
       grad_fn=<MaskedFillBackward0>)

**스케일드 점곱 어텐션 함수를 수정해 마스킹 동작 추가하기**

In [26]:
def scaled_dot_product_attention(query, key, value, mask=None):
  dim_k = query.size(-1)
  scores = torch.bmm(query, key.transpose(1, 2)) / sqrt(dim_k)

  if mask is not None:
    scores = scores.masked_fill(mask == 0, float("-inf"))

  weights = F.softmax(scores, dim=-1)
  return weights.bmm(value)